In [1]:
#!pip install georasters
#!pip install geoindex
#!pip install spark
#!pip install pyspark

In [2]:
import pandas as pd
import georasters as gr
import matplotlib.pyplot as plt
from geoindex import GeoGridIndex, GeoPoint
import os
from tqdm import tqdm, tqdm_pandas

In [3]:
pd.set_option('display.max_columns', 100)

## Load Water Data

In [4]:
os.getcwd()

'/Users/alexjamar/Documents/Berkeley/Capstone/Capstone_Water_Access/Population'

In [5]:
water_data = pd.read_csv("/Users/alexjamar/Documents/Berkeley/Capstone/Capstone_Water_Access//data/Water_Point_Data_Exchange_Complete_Dataset.csv")

/Users/alexjamar/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2,5,7,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
water_data.head()

,Row ID,#country_name,#water_source,#water_tech,#status_id,#management,#pay,#installer,#install_year,#status,#source,#adm1,#adm2,#wpdx_id,#report_date,#country_id,#activity_id,#data_lnk,#orig_lnk,photo_lnk,#converted,#created,#updated,#lat_deg,#lon_deg,Location,Count,#fecal_coliform_presence,#fecal_coliform_value,#subjective_quality
0,1,"Tanzania, United Republic of",Spring,Gravity Communal standpipe,no,NaN,Never pay,RWE,1972.0,Status:Not functional|Quantity:Dry|Quality:Soft,SNV,Arusha,Longido,wpdx-00000001,01/05/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.83257,36.3090,"(-2.83257, 36.309)",1,NaN,NaN,NaN
1,2,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Breakdown Year:2007|Reason N...,SNV,Arusha,Longido,wpdx-00000002,01/01/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.84781,36.5284,"(-2.84781, 36.5284)",1,NaN,NaN,NaN
2,3,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Quantity:Seasonal|Quality:Soft,SNV,Arusha,Longido,wpdx-00000003,01/01/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.84796,36.5288,"(-2.84796, 36.5288)",1,NaN,NaN,NaN
3,4,"Tanzania, United Republic of",Rainwater harvesting,Gravity Communal standpipe,yes,NaN,Never pay,World Vision,2005.0,Status:Functional|Quantity:Seasonal|Quality:Soft,SNV,Arusha,Longido,wpdx-00000004,01/01/2008 12:00:00 AM +0000,TZ,NaN,http://www.dropbox.com/s/50779gal9v4z4y6/TZ%20...,NaN,NaN,"""#water_tech, #country_id, #data_lnk""",10/23/2015 01:03:07 AM +0000,10/23/2015 01:03:07 AM +0000,-2.85318,36.5276,"(-2.85318, 36.5276)",1,NaN,NaN,NaN
4,5,Nicaragua,NaN,NaN,yes,NaN,NaN,NaN,2011.0,NaN,El Porvenier,Matagalpa,Terrabona,wpdx-00000005,01/01/2011 12:00:00 AM +0000,NI,NaN,http://www.dropbox.com/s/0f70mln5roxhy29/ProjT...,https://www.waterpointdata.org/dataset/resourc...,NaN,"#data_lnk, #lon_deg, #lat_deg, #country_id",04/08/2016 08:35:49 AM +0000,04/08/2016 08:35:49 AM +0000,12.74070,-86.0121,"(12.7407, -86.0121)",1,NaN,NaN,NaN


In [7]:
afghanistan_water = water_data[water_data['#country_name'] == 'Afghanistan']

In [9]:
afghanistan_water.head()

,Row ID,#country_name,#water_source,#water_tech,#status_id,#management,#pay,#installer,#install_year,#status,#source,#adm1,#adm2,#wpdx_id,#report_date,#country_id,#activity_id,#data_lnk,#orig_lnk,photo_lnk,#converted,#created,#updated,#lat_deg,#lon_deg,Location,Count,#fecal_coliform_presence,#fecal_coliform_value,#subjective_quality
82,400,Afghanistan,NaN,DUG WELL,yes,NaN,NaN,DACAAR,2001.0,OK,DACAAR,Khost,Musa Khel,wpdx-00000400,11/16/2006 12:00:00 AM +0000,AF,NaN,http://www.dropbox.com/s/3ns28lb7x8acr4m/DACAA...,NaN,NaN,"#status_id, #source, #country_id, #data_lnk",10/23/2015 01:03:12 AM +0000,10/23/2015 01:03:12 AM +0000,33.5244,69.7361,"(33.5244, 69.7361)",1,NaN,NaN,NaN
133,401,Afghanistan,NaN,DUG WELL,yes,NaN,NaN,DACAAR,2001.0,OK,DACAAR,Khost,Spera,wpdx-00000401,10/27/2003 12:00:00 AM +0000,AF,NaN,http://www.dropbox.com/s/3ns28lb7x8acr4m/DACAA...,NaN,NaN,"#status_id, #source, #country_id, #data_lnk",10/23/2015 01:03:12 AM +0000,10/23/2015 01:03:12 AM +0000,33.1229,69.4120,"(33.1229, 69.412)",1,NaN,NaN,NaN
390,389,Afghanistan,NaN,TUBE WELL,yes,NaN,NaN,DACAAR,2002.0,OK,DACAAR,Khost,Nadir Shah Kot,wpdx-00000389,05/21/2005 12:00:00 AM +0000,AF,NaN,http://www.dropbox.com/s/3ns28lb7x8acr4m/DACAA...,NaN,NaN,"#status_id, #source, #country_id, #data_lnk",10/23/2015 01:03:12 AM +0000,10/23/2015 01:03:12 AM +0000,33.4084,69.6479,"(33.4084, 69.6479)",1,NaN,NaN,NaN
391,390,Afghanistan,NaN,DUG WELL,yes,NaN,NaN,DACAAR,2000.0,OK,DACAAR,Khost,Sabari,wpdx-00000390,11/18/2003 12:00:00 AM +0000,AF,NaN,http://www.dropbox.com/s/3ns28lb7x8acr4m/DACAA...,NaN,NaN,"#status_id, #source, #country_id, #data_lnk",10/23/2015 01:03:12 AM +0000,10/23/2015 01:03:12 AM +0000,33.3647,69.9561,"(33.3647, 69.9561)",1,NaN,NaN,NaN
392,391,Afghanistan,NaN,DUG WELL,yes,NaN,NaN,DACAAR,2002.0,OK,DACAAR,Khost,Tere Zayi,wpdx-00000391,09/01/2005 12:00:00 AM +0000,AF,NaN,http://www.dropbox.com/s/3ns28lb7x8acr4m/DACAA...,NaN,NaN,"#status_id, #source, #country_id, #data_lnk",10/23/2015 01:03:12 AM +0000,10/23/2015 01:03:12 AM +0000,33.4361,70.0696,"(33.4361, 70.0696)",1,NaN,NaN,NaN


## Radius-based Search

In [15]:
afghanistan_pop = pd.read_csv("Afghanistan_pop_data.csv")

In [16]:
#Create Geo Index of Afghanistan Population Data
geo_index = GeoGridIndex()
for index, row in afghanistan_pop.iterrows():
    geo_index.add_point(GeoPoint(row['y'], row['x'], ref=row['value']))

In [11]:
#Create Geo Index of Afghanistan Water Point Data
geo_index_water = GeoGridIndex()
for index, row in afghanistan_water.iterrows():
    try:
        geo_index_water.add_point(GeoPoint(row['#lat_deg'], row['#lon_deg'], ref=row['#status_id']))
    except:
        print("Row ID:", row['Row ID'])
        print("Lat:", row['#lat_deg'])
        print("Long:", row['#lon_deg'])

In [17]:
#Calculate population within x distance of water point
def calculate_population_within_x_km(row, index, km):
    center_point = GeoPoint(row['#lat_deg'], row['#lon_deg'])
    total_population = 0
    try:
        for point,distance in index.get_nearest_points(center_point, km, 'km'):
            total_population += point.ref
    except:
        print("Invalid data - Record skipped")
    #print("Total population within", km, "kilometers:", int(total_population))
    return total_population

In [18]:
#Calculate # of water points within x distance of water point
def calculate_water_points_within_x_km(row, index, km):
    center_point = GeoPoint(row['#lat_deg'], row['#lon_deg'])
    total_water_points = 0
    total_functioning_water_points = 0
    try:
        for point,distance in index.get_nearest_points(center_point, km, 'km'):
            total_water_points += 1
            if point.ref == 'yes':
                total_functioning_water_points += 1
    except:
        print("Invalid data - Record skipped")
        
    #Adjust for self
    total_water_points -= 1
    if row['#status_id'] == 'yes':
        total_functioning_water_points -= 1
    return (total_water_points, total_functioning_water_points)

In [19]:
#Add 1 km radius population data to each well
tqdm.pandas()
afghanistan_water['1 km population'] = afghanistan_water.progress_apply(lambda row: calculate_population_within_x_km(row, geo_index, 1),
                                                  axis = 1)

100%|██████████| 59849/59849 [71:07:51<00:00,  4.51s/it]   
/Users/alexjamar/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [14]:
#Add 1 km radius water point data to each water point
tqdm.pandas()
afghanistan_water['1 km water points (total, functioning)'] = afghanistan_water.progress_apply(lambda row: calculate_water_points_within_x_km(row, geo_index_water, 1),
                                                  axis = 1)

100%|██████████| 59849/59849 [17:34<00:00, 62.40it/s]  
/Users/alexjamar/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [20]:
afghanistan_water.head(2)

,Row ID,#country_name,#water_source,#water_tech,#status_id,#management,#pay,#installer,#install_year,#status,#source,#adm1,#adm2,#wpdx_id,#report_date,#country_id,#activity_id,#data_lnk,#orig_lnk,photo_lnk,#converted,#created,#updated,#lat_deg,#lon_deg,Location,Count,#fecal_coliform_presence,#fecal_coliform_value,#subjective_quality,"1 km water points (total, functioning)",1 km population
82,400,Afghanistan,NaN,DUG WELL,yes,NaN,NaN,DACAAR,2001.0,OK,DACAAR,Khost,Musa Khel,wpdx-00000400,11/16/2006 12:00:00 AM +0000,AF,NaN,http://www.dropbox.com/s/3ns28lb7x8acr4m/DACAA...,NaN,NaN,"#status_id, #source, #country_id, #data_lnk",10/23/2015 01:03:12 AM +0000,10/23/2015 01:03:12 AM +0000,33.5244,69.7361,"(33.5244, 69.7361)",1,NaN,NaN,NaN,"(2, 1)",641.691
133,401,Afghanistan,NaN,DUG WELL,yes,NaN,NaN,DACAAR,2001.0,OK,DACAAR,Khost,Spera,wpdx-00000401,10/27/2003 12:00:00 AM +0000,AF,NaN,http://www.dropbox.com/s/3ns28lb7x8acr4m/DACAA...,NaN,NaN,"#status_id, #source, #country_id, #data_lnk",10/23/2015 01:03:12 AM +0000,10/23/2015 01:03:12 AM +0000,33.1229,69.4120,"(33.1229, 69.412)",1,NaN,NaN,NaN,"(0, 0)",378.520


In [21]:
def clean_water_data(df):
    df['1 km population'] = df['1 km population'].apply(lambda x: int(round(x,0)))
    df['1 km total water points'] = df['1 km water points (total, functioning)'].apply(lambda x: str(x).split(',')[0][1:])
    df['1 km functioning water points'] = df['1 km water points (total, functioning)'].apply(lambda x: str(x).split(',')[1][:-1])
    df = df[['Row ID', '#country_name', '#status_id', '#lat_deg', '#lon_deg', '1 km population',
                        '1 km total water points', '1 km functioning water points']]
    
    return df

In [22]:
afghanistan_abridged = clean_water_data(afghanistan_water)

/Users/alexjamar/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/alexjamar/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/alexjamar/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [23]:
afghanistan_abridged.head()

,Row ID,#country_name,#status_id,#lat_deg,#lon_deg,1 km population,1 km total water points,1 km functioning water points
82,400,Afghanistan,yes,33.5244,69.7361,642,2,1
133,401,Afghanistan,yes,33.1229,69.4120,379,0,0
390,389,Afghanistan,yes,33.4084,69.6479,174,0,0
391,390,Afghanistan,yes,33.3647,69.9561,995,23,19
392,391,Afghanistan,yes,33.4361,70.0696,890,13,10


In [24]:
afghanistan_abridged.to_csv("Abridged_Afghanistan_Water_and_Population_Data.csv", index = False)